# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

Also, to avoid name error during variable assignment, we need to install and import PyTorch (https://pytorch.org/get-started/locally/#windows-pip).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import random
import torch

In [2]:
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [3]:
#my local machine runs on Windows 64-bit and the environment executable file was placed in this repo after setting up
authorLocalPath = r".\Food-Collector-Windows-64\UnityEnvironment.exe"
env = UnityEnvironment(file_name=authorLocalPath)

UnityTimeOutException: The Unity environment took too long to respond. Make sure that :
	 The environment does not need user interaction to launch
	 The Academy and the External Brain(s) are attached to objects in the Scene
	 The environment and the Python interface have compatible versions.

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
print(brain)

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The environment state is an array of raw pixels with shape (1, 84, 84, 3). Note that this code differs from the notebook for the project, where we are grabbing visual_observations (the raw pixels) instead of vector_observations. A reward of +1 is provided for collecting a yellow banana, and a reward of -1 is provided for collecting a blue banana.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.visual_observations[0]
print('States look like:')
plt.imshow(np.squeeze(state))
plt.show()
state_size = state.shape
print('States have shape:', state.shape)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
# env_info = env.reset(train_mode=True)[brain_name] # reset the environment
# state = env_info.vector_observations[0]            # get the current state
# score = 0 # initialize the score
# step_t = 0 # track time step
# while True:
#     action = np.random.randint(action_size)        # select an action
#     step_t+=1
#     env_info = env.step(action)[brain_name]        # send the action to the environment
#     next_state = env_info.visual_observations[0]   # get the next state
#     reward = env_info.rewards[0]                   # get the reward
#     done = env_info.local_done[0]                  # see if episode has finished
#     score += reward                                # update the score
#     state = next_state                             # roll over the state to next time step
#     if done:                                       # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))
# print("Terminal time step: {}".format(step_t))

### 4. Training the agent

In [ ]:
N_EPISODES = 300
MIN_EPISODES = 10
MAX_T = 300
PERF_BENCHMARK = 5

In [ ]:
#Import dqn_agent.py, which also in turn import model.py
from agent import DQNAgent
env_info = env.reset(train_mode=True)[brain_name]
agent = DQNAgent(state_size=state_size, action_size=action_size, seed=0)

def dqn_train(n_episodes=N_EPISODES, max_t=MAX_T, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)   # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.visual_observations[0]
        state = agent.preprocess_state(state)
        state = agent.stack_images(state,state)
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps) #determine the action based on epsilon-greedy policy
            env_info = env.step(np.int32(action))[brain_name]        # send the action to the environment
            next_state = env_info.visual_observations[0]   # get the next state
            next_state = agent.preprocess_state(next_state) # turn theimage to greyscale
            next_state = agent.stack_images(next_state, state) #stack the next state with current state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0] 
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=PERF_BENCHMARK and i_episode > MIN_EPISODES:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-1, np.mean(scores_window)))
            agent.save_checkpoints(mean_score=np.mean(scores_window))
            break
    return scores

scores = dqn_train()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 5. Watch a Smart Agent!

In the next code cell, we will load the trained weights from file to watch a smart agent!

In [ ]:
# # load the weights from file
# agent.load_checkpoints(mean_score=int(13))

# for i in range(3):
#     env_info = env.reset(train_mode=False)[brain_name]
#     state = env_info.vector_observations[0] 
#     score = 0
#     for j in range(300):
#         action = agent.act(state)
#         env_info = env.step(np.int32(action))[brain_name]        # send the action to the environment
#         reward = env_info.rewards[0]                   # get the reward
#         done = env_info.local_done[0]
#         next_state = env_info.vector_observations[0] # get the next state
#         state = next_state
#         score += reward
#         if done:
#             print('\rRun: {} , Score: {}'.format(i+1,score))
#             break 
# # Uncomment the below line to close the environment at the end      
# # env.close()